# Launch Sites Locations Analysis

The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.

Previously, we did some exploratory data analysis by visualizing the SpaceX launch dataset using matplotlib and seaborn and we discovered some preliminary correlations between the launch site and success rates. Here, we will be displaying interactive geospacial visual analytics using Folium.

## Objectives
- Mark all launch sites on a map
- Mark the success/failed launches for each site on the map
- Calculate the distances between a launch site to its proximities

Through these visualizations, we should be able to find some geographical patterns about launch sites.

### Importing Dependencies

In [2]:
import pandas as pd

import folium
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

import wget

### Loading Dataset

> The following dataset with the name spacex_launch_geo.csv is an augmented dataset with latitude and longitude added for each site.

In [3]:
spacex_df = pd.read_csv('Data/spacex_launch_geo.csv')
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Mission Outcome,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Success,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),Success,No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),Success,No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),Success,No attempt,0,28.562302,-80.577356


### Marking all launch sites on a map
First, let's try to add each site's location on a map using site's latitude and longitude coordinates

In [4]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launchsites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launchsites_df = launchsites_df[['Launch Site', 'Lat', 'Long']]
launchsites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610746


The above coordinates are just plain numbers that can not give us any intuitive insights about where those launch sites are. If one is used to dealing with geographic coordinates, one could interpret those numbers directly in their mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.

We first start by creating a folium Map object, with an initial center location to be NASA Johnson Space Center at Houston, Texas. This first close up is for demonstrative purposes only.

In [5]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [6]:
# Create a blue circle at NASA Space Center's Co-ordinate with a popup label showing its name.
circle = folium.Circle(nasa_coordinate, radius=1000, color='#008eed', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))

# Create a blue circle at NASA Space Center's Co-ordinate with a icon showing its name
marker = folium.Marker(
    nasa_coordinate,
    # Create a icon as a text label
    icon= DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color: #008eed;"><b>%s</b></div>'%'NASA JSC'
    )
)

site_map.add_child(circle)
site_map.add_child(marker)
site_map

Let's add a circle for each launch site in data frame <code>launch_sites</code>

In [8]:
# Initial the Map
site_map = folium.Map(location=nasa_coordinate, zoom_start=4)

# defining a function for each launch site, add a circle object based on its co-ordinate values as well as a popup label.
def add_launch_sites(row):
    circle = folium.Circle([launchsites_df['Lat'][row], launchsites_df['Long'][row]], radius=50, color='#d35400', fill=True).add_child(folium.Popup(launchsites_df['Launch Site'][row]))
    marker = folium.Marker([launchsites_df['Lat'][row], launchsites_df['Long'][row]],
    
        # create an icon as text label
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html = '<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % launchsites_df['Launch Site'][row] 
            )
    )
    site_map.add_child(circle)
    site_map.add_child(marker)

for row in range(len(launchsites_df)):
    add_launch_sites(row)

site_map